In [1]:
# Importing Libraries: data loading and EDA
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Ignoring warnings
import warnings
warnings.filterwarnings('ignore')

# Importing libraries for text preprocessing
import re,spacy,nltk,string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS


In [2]:
df=pd.read_csv('tickets.csv')

In [3]:
df

,Unnamed: 0,tags,zip_code,complaint_id,issue,date_received,state,consumer_disputed,product,company_response,company,submitted_via,date_sent_to_company,company_public_response,sub_product,timely,complaint_what_happened,sub_issue,consumer_consent_provided
0,0,NaN,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,NaN,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,NaN,Credit card debt,Yes,NaN,Debt is not yours,Consent not provided
1,1,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,NaN,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,NaN,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,2,NaN,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,NaN,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,3,NaN,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,NaN,Mortgage,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,NaN,Conventional home mortgage,Yes,NaN,NaN,Consent not provided
4,4,NaN,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,NaN,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,NaN,Charged too much interest,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78308,78308,NaN,8857,3105617,Closing an account,2018-12-20T12:00:00-05:00,NJ,NaN,Checking or savings account,Closed with explanation,JPMORGAN CHASE & CO.,Web,2018-12-21T12:00:00-05:00,NaN,Checking account,Yes,NaN,Funds not received from closed account,Consent not provided
78309,78309,NaN,934XX,3091984,"Other features, terms, or problems",2018-12-05T12:00:00-05:00,CA,NaN,Credit card or prepaid card,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2018-12-05T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",Other problem,Consent provided
78310,78310,NaN,923XX,3133355,Problem with a lender or other company chargin...,2019-01-25T12:00:00-05:00,CA,NaN,Checking or savings account,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-01-25T12:00:00-05:00,NaN,Checking account,Yes,I am not familiar with XXXX pay and did not un...,Transaction was not authorized,Consent provided
78311,78311,NaN,NaN,3110963,"Other features, terms, or problems",2018-12-27T12:00:00-05:00,PA,NaN,Credit card or prepaid card,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2018-12-27T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,I have had flawless credit for 30 yrs. I've ha...,Problem with balance transfer,Consent provided


In [4]:
df.head()

,Unnamed: 0,tags,zip_code,complaint_id,issue,date_received,state,consumer_disputed,product,company_response,company,submitted_via,date_sent_to_company,company_public_response,sub_product,timely,complaint_what_happened,sub_issue,consumer_consent_provided
0,0,NaN,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,NaN,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,NaN,Credit card debt,Yes,NaN,Debt is not yours,Consent not provided
1,1,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,NaN,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,NaN,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,2,NaN,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,NaN,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,3,NaN,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,NaN,Mortgage,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,NaN,Conventional home mortgage,Yes,NaN,NaN,Consent not provided
4,4,NaN,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,NaN,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,NaN,Charged too much interest,NaN


In [5]:
df.shape

(78313, 19)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78313 entries, 0 to 78312
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 78313 non-null  int64 
 1   tags                       10900 non-null  object
 2   zip_code                   71556 non-null  object
 3   complaint_id               78313 non-null  int64 
 4   issue                      78313 non-null  object
 5   date_received              78313 non-null  object
 6   state                      76322 non-null  object
 7   consumer_disputed          42630 non-null  object
 8   product                    78313 non-null  object
 9   company_response           78313 non-null  object
 10  company                    78313 non-null  object
 11  submitted_via              78313 non-null  object
 12  date_sent_to_company       78313 non-null  object
 13  company_public_response    4 non-null      object
 14  sub_pr

In [7]:
df.isna().sum()

Unnamed: 0                       0
tags                         67413
zip_code                      6757
complaint_id                     0
issue                            0
date_received                    0
state                         1991
consumer_disputed            35683
product                          0
company_response                 0
company                          0
submitted_via                    0
date_sent_to_company             0
company_public_response      78309
sub_product                  10571
timely                           0
complaint_what_happened      57241
sub_issue                    46297
consumer_consent_provided    43855
dtype: int64

In [8]:
del df['tags']

In [9]:
df = df.rename(columns={'Unnamed: 0': 'Index'})

In [10]:
df

,Index,zip_code,complaint_id,issue,date_received,state,consumer_disputed,product,company_response,company,submitted_via,date_sent_to_company,company_public_response,sub_product,timely,complaint_what_happened,sub_issue,consumer_consent_provided
0,0,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,NaN,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,NaN,Credit card debt,Yes,NaN,Debt is not yours,Consent not provided
1,1,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,NaN,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,NaN,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,2,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,NaN,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,3,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,NaN,Mortgage,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,NaN,Conventional home mortgage,Yes,NaN,NaN,Consent not provided
4,4,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,NaN,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,NaN,Charged too much interest,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78308,78308,8857,3105617,Closing an account,2018-12-20T12:00:00-05:00,NJ,NaN,Checking or savings account,Closed with explanation,JPMORGAN CHASE & CO.,Web,2018-12-21T12:00:00-05:00,NaN,Checking account,Yes,NaN,Funds not received from closed account,Consent not provided
78309,78309,934XX,3091984,"Other features, terms, or problems",2018-12-05T12:00:00-05:00,CA,NaN,Credit card or prepaid card,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2018-12-05T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",Other problem,Consent provided
78310,78310,923XX,3133355,Problem with a lender or other company chargin...,2019-01-25T12:00:00-05:00,CA,NaN,Checking or savings account,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-01-25T12:00:00-05:00,NaN,Checking account,Yes,I am not familiar with XXXX pay and did not un...,Transaction was not authorized,Consent provided
78311,78311,NaN,3110963,"Other features, terms, or problems",2018-12-27T12:00:00-05:00,PA,NaN,Credit card or prepaid card,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2018-12-27T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,I have had flawless credit for 30 yrs. I've ha...,Problem with balance transfer,Consent provided


In [11]:
df.index=df['Index']
del df['Index']

In [12]:
df.head()

,zip_code,complaint_id,issue,date_received,state,consumer_disputed,product,company_response,company,submitted_via,date_sent_to_company,company_public_response,sub_product,timely,complaint_what_happened,sub_issue,consumer_consent_provided
Index,,,,,,,,,,,,,,,,,
0,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,NaN,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,NaN,Credit card debt,Yes,NaN,Debt is not yours,Consent not provided
1,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,NaN,Debt collection,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,NaN,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,NaN,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,NaN,Mortgage,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,NaN,Conventional home mortgage,Yes,NaN,NaN,Consent not provided
4,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,NaN,Credit card or prepaid card,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,NaN,General-purpose credit card or charge card,Yes,NaN,Charged too much interest,NaN


In [13]:
cols=df.columns
cols

Index(['zip_code', 'complaint_id', 'issue', 'date_received', 'state',
       'consumer_disputed', 'product', 'company_response', 'company',
       'submitted_via', 'date_sent_to_company', 'company_public_response',
       'sub_product', 'timely', 'complaint_what_happened', 'sub_issue',
       'consumer_consent_provided'],
      dtype='object')

In [14]:
# Converting all the columns of object datatype to lower case
for col in cols:
    if df[col].dtype == 'object':
        df[col] = df[col].str.lower()

In [15]:
df

,zip_code,complaint_id,issue,date_received,state,consumer_disputed,product,company_response,company,submitted_via,date_sent_to_company,company_public_response,sub_product,timely,complaint_what_happened,sub_issue,consumer_consent_provided
Index,,,,,,,,,,,,,,,,,
0,90301,3211475,attempts to collect debt not owed,2019-04-13t12:00:00-05:00,ca,NaN,debt collection,closed with explanation,jpmorgan chase & co.,web,2019-04-13t12:00:00-05:00,NaN,credit card debt,yes,NaN,debt is not yours,consent not provided
1,319xx,3229299,written notification about debt,2019-05-01t12:00:00-05:00,ga,NaN,debt collection,closed with explanation,jpmorgan chase & co.,web,2019-05-01t12:00:00-05:00,NaN,credit card debt,yes,good morning my name is xxxx xxxx and i apprec...,didn't receive enough information to verify debt,consent provided
2,77069,3199379,"other features, terms, or problems",2019-04-02t12:00:00-05:00,tx,NaN,credit card or prepaid card,closed with explanation,jpmorgan chase & co.,web,2019-04-02t12:00:00-05:00,NaN,general-purpose credit card or charge card,yes,i upgraded my xxxx xxxx card in xx/xx/2018 and...,problem with rewards from credit card,consent provided
3,48066,2673060,trouble during payment process,2017-09-13t12:00:00-05:00,mi,NaN,mortgage,closed with explanation,jpmorgan chase & co.,web,2017-09-14t12:00:00-05:00,NaN,conventional home mortgage,yes,NaN,NaN,consent not provided
4,10473,3203545,fees or interest,2019-04-05t12:00:00-05:00,ny,NaN,credit card or prepaid card,closed with explanation,jpmorgan chase & co.,referral,2019-04-05t12:00:00-05:00,NaN,general-purpose credit card or charge card,yes,NaN,charged too much interest,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78308,8857,3105617,closing an account,2018-12-20t12:00:00-05:00,nj,NaN,checking or savings account,closed with explanation,jpmorgan chase & co.,web,2018-12-21t12:00:00-05:00,NaN,checking account,yes,NaN,funds not received from closed account,consent not provided
78309,934xx,3091984,"other features, terms, or problems",2018-12-05t12:00:00-05:00,ca,NaN,credit card or prepaid card,closed with monetary relief,jpmorgan chase & co.,web,2018-12-05t12:00:00-05:00,NaN,general-purpose credit card or charge card,yes,"on wednesday, xx/xx/xxxx i called chas, my xxx...",other problem,consent provided
78310,923xx,3133355,problem with a lender or other company chargin...,2019-01-25t12:00:00-05:00,ca,NaN,checking or savings account,closed with explanation,jpmorgan chase & co.,web,2019-01-25t12:00:00-05:00,NaN,checking account,yes,i am not familiar with xxxx pay and did not un...,transaction was not authorized,consent provided


In [16]:
# Dropping rows with NULL values in column complaint_what_happened
if df['complaint_what_happened'].isna().any():
  df.dropna(subset=['complaint_what_happened'], inplace=True)

In [17]:
df['complaint_what_happened'].isna().sum()

0

In [18]:
df['complaint_what_happened'] = df['complaint_what_happened'].astype(str)

In [19]:
df['complaint_what_happened'].dtype

dtype('O')

In [21]:
import contractions
def expand_contractions(text):
   expanded_text = contractions.fix(text)
   return expanded_text

In [22]:
df['complaint_what_happened'] = df['complaint_what_happened'].apply(lambda x: expand_contractions(x))

In [23]:
df['complaint_what_happened']

Index
1        good morning my name is xxxx xxxx and i apprec...
2        i upgraded my xxxx xxxx card in xx/xx/2018 and...
10       chase card was reported on xx/xx/2019. however...
11       on xx/xx/2018, while trying to book a xxxx  xx...
14       my grand son give me check for {$1600.00} i de...
                               ...                        
78303    after being a chase card customer for well ove...
78309    on wednesday, xx/xx/xxxx i called chas, my xxx...
78310    i am not familiar with xxxx pay and did not un...
78311    i have had flawless credit for 30 yrs. i have ...
78312    roughly 10+ years ago i closed out my accounts...
Name: complaint_what_happened, Length: 21072, dtype: object

In [24]:
# Using regular expressions to find the patterns
# Removing the text enclosed within square brackets
df['complaint_what_happened'] = df['complaint_what_happened'].apply(lambda x: re.sub(r'\[.*?\]', '', x))

In [25]:
# Removing the punctuations
import string
exclude=string.punctuation
remove_punctuation=str.maketrans('','',exclude)
df['complaint_what_happened'] = df['complaint_what_happened'].apply(lambda x:x.translate(remove_punctuation))

In [26]:
# Checking if the punctuations are removed
punctuation_pattern = f"[{re.escape(string.punctuation)}]"
df1 = df['complaint_what_happened'].str.contains(punctuation_pattern)
df1.unique()

array([False])

In [27]:
df['complaint_what_happened'].iloc[1]

'i upgraded my xxxx xxxx card in xxxx2018 and was told by the agent who did the upgrade my anniversary date would not change it turned the agent was giving me the wrong information in order to upgrade the account xxxx   changed my anniversary date from xxxxxxxx to xxxxxxxx without my consent xxxx has the recording of the agent who was misled me'

In [28]:
pattern = r'xxxx xxxx|xxxx2018|xxxx|xxxxxxxx'

# Remove the specified phrases from the 'complaint_what_happened' column
df['complaint_what_happened'] = df['complaint_what_happened'].str.replace(pattern, '', regex=True)

In [29]:
df['complaint_what_happened'].iloc[1]

'i upgraded my  card in  and was told by the agent who did the upgrade my anniversary date would not change it turned the agent was giving me the wrong information in order to upgrade the account    changed my anniversary date from  to  without my consent  has the recording of the agent who was misled me'

In [30]:
# Tokenization
from nltk.tokenize import word_tokenize, sent_tokenize
df['tokenized_complaint'] = df['complaint_what_happened'].apply(lambda x: word_tokenize(x))
df['tokenized_complaint']

Index
1        [good, morning, my, name, is, and, i, apprecia...
2        [i, upgraded, my, card, in, and, was, told, by...
10       [chase, card, was, reported, on, 2019, however...
11       [on, while, trying, to, book, a, ticket, i, ca...
14       [my, grand, son, give, me, check, for, 160000,...
                               ...                        
78303    [after, being, a, chase, card, customer, for, ...
78309    [on, wednesday, i, called, chas, my, visa, cre...
78310    [i, am, not, familiar, with, pay, and, did, no...
78311    [i, have, had, flawless, credit, for, 30, yrs,...
78312    [roughly, 10, years, ago, i, closed, out, my, ...
Name: tokenized_complaint, Length: 21072, dtype: object

## Removing Stopwords

In [31]:
from nltk.corpus import stopwords

In [32]:
stop_words = set(stopwords.words('english'))

In [33]:
def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens

df['tokenized_stopword_removed'] = df['tokenized_complaint'].apply(remove_stopwords)

In [34]:
df['tokenized_stopword_removed']

Index
1        [good, morning, name, appreciate, could, help,...
2        [upgraded, card, told, agent, upgrade, anniver...
10       [chase, card, reported, 2019, however, fraudul...
11       [trying, book, ticket, came, across, offer, 30...
14       [grand, son, give, check, 160000, deposit, cha...
                               ...                        
78303    [chase, card, customer, well, decade, offered,...
78309    [wednesday, called, chas, visa, credit, card, ...
78310    [familiar, pay, understand, great, risk, provi...
78311    [flawless, credit, 30, yrs, chase, credit, car...
78312    [roughly, 10, years, ago, closed, accounts, jp...
Name: tokenized_stopword_removed, Length: 21072, dtype: object

## Joining the resulting list from stopword removal into a string and putting it in a different column -- helps remove any extra spaces in the middle

In [35]:
df['complaint_1'] = df['tokenized_stopword_removed'].apply(lambda x: ' '.join(x))

In [36]:
df.loc[1, 'complaint_1']

'good morning name appreciate could help put stop chase bank cardmember services 2018 wrote chase asking debt verification sent statement acceptable asking bank validate debt instead receiving mail every month attempting collect debt right know information consumer chase account thanks advance help'

## Lemmatizing the resulting string (after stopwords removal) 

In [37]:
#Lemmetization
from nltk.stem import WordNetLemmatizer
import spacy
nlp = spacy.load("en_core_web_md")

In [38]:
lemmatized_list=[]
for text in df['complaint_1']:
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    lemmatized_list.append(lemmatized_text)
df['lemmatized_complaint'] = lemmatized_list

lemmatized_list=[]
for text in df['complaint_what_happened']:
    doc = nlp(text)
    lemmatized_text = []
    for token in doc:
        lemmatized_text.append(token.lemma_)
    lemmatized_text = ' '.join(lemmatized_text)
    lemmatized_list.append(lemmatized_text)
    
    

In [39]:
df.loc[1, 'lemmatized_complaint']

'good morning name appreciate could help put stop chase bank cardmember service 2018 write chase ask debt verification send statement acceptable ask bank validate debt instead receive mail every month attempt collect debt right know information consumer chase account thank advance help'

In [40]:
df['lemmatized_complaint']

Index
1        good morning name appreciate could help put st...
2        upgrade card tell agent upgrade anniversary da...
10       chase card report 2019 however fraudulent appl...
11       try book ticket come across offer 30000 apply ...
14       grand son give check 160000 deposit chase acco...
                               ...                        
78303    chase card customer well decade offer multiple...
78309    wednesday call chas visa credit card provider ...
78310    familiar pay understand great risk provide con...
78311    flawless credit 30 yrs chase credit card chase...
78312    roughly 10 year ago close account jp morgan ch...
Name: lemmatized_complaint, Length: 21072, dtype: object

## POS Tagging -- just taking out Nouns, Proper Nouns and Adjectives from the cleaned complaint texts

In [42]:
def extract_nouns_adj(text):
    doc = nlp(text)
    nouns = [token.text for token in doc if token.pos_ in ('ADJ', 'NOUN', 'PROPN')]
    return " ".join(nouns)

In [43]:
df['POS_NN_PRN_JJ'] = df['lemmatized_complaint'].apply(lambda x: extract_nouns_adj(x))

In [44]:
df[['lemmatized_complaint','POS_NN_PRN_JJ']]

,lemmatized_complaint,POS_NN_PRN_JJ
Index,,
1,good morning name appreciate could help put st...,good morning name stop chase bank cardmember s...
2,upgrade card tell agent upgrade anniversary da...,upgrade card agent upgrade anniversary date ag...
10,chase card report 2019 however fraudulent appl...,chase card report fraudulent application ident...
11,try book ticket come across offer 30000 apply ...,book ticket offer ticket reward card informati...
14,grand son give check 160000 deposit chase acco...,grand son check deposit chase account fund cle...
...,...,...
78303,chase card customer well decade offer multiple...,chase card customer decade multiple solicitati...
78309,wednesday call chas visa credit card provider ...,wednesday call chas visa credit card provider ...
78310,familiar pay understand great risk provide con...,familiar pay great risk consumer safe chase ba...


## TF-IDF Vectorizer -- considers the importance of words based on how often they occur

### TF-IDF assigns weights to words based on their importance within individual documents and across the entire corpus

In [45]:
vect = TfidfVectorizer(min_df=2, max_df=0.90, stop_words='english')

In [46]:
X = vect.fit_transform(df['POS_NN_PRN_JJ'])

## Topic Modelling -- LDA

In [47]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.model_selection import GridSearchCV

In [48]:
lda = LDA(n_components = 5, learning_method="batch", max_iter=50, random_state=0)

document_topics = lda.fit_transform(X)

In [50]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vect.get_feature_names_out())

In [51]:
import mglearn
mglearn.tools.print_topics(topics=range(5), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=40)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
loan          account       credit        credit        flight        
mortgage      chase         card          report        ticket        
chase         bank          chase         dispute       airline       
payment       check         payment       charge        book          
home          money         account       chase         travel        
modification  deposit       balance       card          trip          
property      fund          fee           inquiry       plane         
year          transaction   late          account       voucher       
bank          day           charge        company       bag           
debt          branch        month         merchant      refund        
time          card          time          information   seat          
foreclosure   claim         report        letter        passenger     
pay   

lda = LatentDirichletAllocation(n_components = 3, learning_method="batch", max_iter=50, random_state=0)

document_topics = lda.fit_transform(X)
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vect.get_feature_names())
import mglearn
mglearn.tools.print_topics(topics=range(3), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=40)

lda = LatentDirichletAllocation(n_components = 4, learning_method="batch", max_iter=50, random_state=0)

document_topics = lda.fit_transform(X)
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vect.get_feature_names())
import mglearn
mglearn.tools.print_topics(topics=range(4), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=40)

lda = LDA(n_components = 6, learning_method="batch", max_iter=50, random_state=0)

document_topics = lda.fit_transform(X)
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vect.get_feature_names())
import mglearn
mglearn.tools.print_topics(topics=range(6), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=40)

lda = LatentDirichletAllocation(n_components = 7, learning_method="batch", max_iter=50, random_state=0)

document_topics = lda.fit_transform(X)
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vect.get_feature_names())
import mglearn
mglearn.tools.print_topics(topics=range(7), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=40)

In [52]:
document_topics_df = pd.DataFrame(document_topics, columns=[f"Topic_{i}" for i in range(5)], index=df.index)

dominant_topic = document_topics_df.idxmax(axis=1)
dominant_topic = dominant_topic.apply(lambda x: int(x.split('_')[1]))  # Extract the numeric part of the column name

df['Dominant_Topic'] = dominant_topic

In [53]:
document_topics_df

,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4
Index,,,,,
1,0.039260,0.039357,0.843553,0.039909,0.037922
2,0.052021,0.052066,0.791987,0.052619,0.051308
10,0.055131,0.054375,0.263838,0.572754,0.053901
11,0.032602,0.032733,0.817324,0.033026,0.084313
14,0.046744,0.814014,0.046586,0.046585,0.046071
...,...,...,...,...,...
78303,0.036103,0.036134,0.848403,0.036588,0.042773
78309,0.373614,0.027043,0.026794,0.546629,0.025921
78310,0.022799,0.685802,0.245648,0.023673,0.022078


In [54]:
df['Dominant_Topic'].isna().sum()

0

In [55]:
df.to_csv("Label_Tickets_5.csv")

In [56]:
for x in range(6):
    print(f"Topic {x} has ", len(df[df['Dominant_Topic']==x]), "data points.")

Topic 0 has  4445 data points.
Topic 1 has  6982 data points.
Topic 2 has  5804 data points.
Topic 3 has  3806 data points.
Topic 4 has  35 data points.
Topic 5 has  0 data points.
